# Raw data -> Reuse distances

In [1]:
import json
import os
import csv
import pandas as pd
import numpy as np

In [ ]:
path = '../raw_data/'
hist = {}
second_hist = {}
fnames = []
for file in os.listdir(path):
    if file[-9:] != '_2048.out':
        continue
    id_ctr = 0
    hist_list = []
    with open(path+file) as f:
        lines = f.readlines()
        ctr = 0
        conf_values = ['id']
        while lines[ctr] != "~~~~~~~~~~Reuse Distance Profiler output~~~~~~~~~~\n":
            ctr += 1
        ctr += 1
        while lines[ctr] != "~~~~~~~~~~Reuse Distance Profiler output end~~~~~~~~~~\n":
            x, y, z = lines[ctr].split()
            conf_values.append(x)
            second_hist[x] = int(y)
            ctr += 1
        while len(lines) - 20 > ctr:
            while lines[ctr] != "~~~~~~~~~~Reuse Distance Profiler output~~~~~~~~~~\n":
                ctr += 1
            hist['id'] = file[:-8] + str(id_ctr)
            id_ctr += 1
            ctr += 1
            while lines[ctr] != "~~~~~~~~~~Reuse Distance Profiler output end~~~~~~~~~~\n":
                x, y, z = lines[ctr].split()
                hist[x] = int(y) - second_hist[x]
                second_hist[x] = int(y)
                ctr += 1
            hist_list.append(hist.copy())
    id_ctr -= 1       # Since we don't keep the last one
    fnames.append(file[:-9])
    print(f'Printing: '+'reuse_distances/'+file[:-9]+'.csv'+f', {len(hist_list)} rds')
    with open('reuse_distances/'+file[:-9]+'.csv', 'w', newline="") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=conf_values)
        writer.writeheader()
        # writer.writerow(second_hist)    # Final/full histogram
        for hst in hist_list[:-1]:     # (last one's all 0 and first one has warmup instructions)
            writer.writerow(hst)        # In between histograms


# Raw data -> Total accesses, misses

In [2]:
import pandas as pd
import numpy as np

In [3]:
path = '../ce_raw_data/ship_llc/'
id_ctr = 0
columns = ['id','1024','2048','4096','8192']
l1d_full = pd.DataFrame()
l2c_full = pd.DataFrame()
llc_full = pd.DataFrame()
l1d_accesses_full = pd.DataFrame()
l2c_accesses_full = pd.DataFrame()
llc_accesses_full = pd.DataFrame()
l1d_misses_full = pd.DataFrame()
l2c_misses_full = pd.DataFrame()
llc_misses_full = pd.DataFrame()
for file in os.listdir(path):
    if file[-9:] != '_2048.out':
        continue
    l1d_small = pd.DataFrame(0., index=np.arange(749), columns=columns)
    l2c_small = pd.DataFrame(0., index=np.arange(749), columns=columns)
    llc_small = pd.DataFrame(0., index=np.arange(749), columns=columns)
    l1d_accesses = pd.DataFrame(0, index=np.arange(749), columns=columns)
    l2c_accesses = pd.DataFrame(0, index=np.arange(749), columns=columns)
    llc_accesses = pd.DataFrame(0, index=np.arange(749), columns=columns)
    l1d_misses = pd.DataFrame(0, index=np.arange(749), columns=columns)
    l2c_misses = pd.DataFrame(0, index=np.arange(749), columns=columns)
    llc_misses = pd.DataFrame(0, index=np.arange(749), columns=columns)
    # print(file)
    for c in columns[1:]:
        fname = file[:-9] + '_' + c + '.out'
        checker = False
        with open(path+fname) as f:
            lines = f.readlines()
            ctr = 0
            prev_misses_l1d = 0.
            prev_accesses_l1d = 0.
            prev_misses_l2c = 0.
            prev_accesses_l2c = 0.
            prev_misses_llc = 0.
            prev_accesses_llc = 0.
            for line in lines:
                if line != "~~~~~~~~~~Reuse Distance Profiler output~~~~~~~~~~\n":
                    checker=True
                if line[:14] == "cpu0_L1D TOTAL":
                    ctr+=1
                    if ctr == 1:
                        prev_misses_l1d = int(line.split()[7])
                        prev_accesses_l1d = int(line.split()[3])
                    if 751 > ctr and ctr > 1:
                        l1d_misses[c][ctr-2] = int(line.split()[7]) - prev_misses_l1d
                        prev_misses_l1d = int(line.split()[7])
                        l1d_accesses[c][ctr-2] = int(line.split()[3]) - prev_accesses_l1d
                        prev_accesses_l1d = int(line.split()[3])
                        if l1d_accesses[c][ctr-2] == 0: continue
                        l1d_small[c][ctr-2] = float(l1d_misses[c][ctr-2])/float(l1d_accesses[c][ctr-2])
                if line[:14] == "cpu0_L2C TOTAL":
                    if ctr == 1:
                        prev_misses_l2c = int(line.split()[7])
                        prev_accesses_l2c = int(line.split()[3])
                    if 751 > ctr and ctr > 1:
                        l2c_misses[c][ctr-2] = int(line.split()[7]) - prev_misses_l2c
                        prev_misses_l2c = int(line.split()[7])
                        l2c_accesses[c][ctr-2] = int(line.split()[3]) - prev_accesses_l2c
                        prev_accesses_l2c = int(line.split()[3])
                        if l2c_accesses[c][ctr-2] == 0: continue
                        l2c_small[c][ctr-2] = float(l2c_misses[c][ctr-2])/float(l2c_accesses[c][ctr-2])
                if line[:9] == "LLC TOTAL":
                    if ctr == 1:
                        prev_misses_llc = int(line.split()[7])
                        prev_accesses_llc = int(line.split()[3])
                    if 751 > ctr and ctr > 1:
                        llc_misses[c][ctr-2] = int(line.split()[7]) - prev_misses_llc
                        prev_misses_llc = int(line.split()[7])
                        llc_accesses[c][ctr-2] = int(line.split()[3]) - prev_accesses_llc
                        prev_accesses_llc = int(line.split()[3])
                        if llc_accesses[c][ctr-2] == 0: continue
                        llc_small[c][ctr-2] = float(llc_misses[c][ctr-2])/float(llc_accesses[c][ctr-2])
            if not checker or ctr != 751:
                print("\n Error:", fname) # Check if a file didn't finish correctly
    print("|",end="")
    # print(l1d_small)
    l1d_small['id'] = file[:-9] + '_' + l1d_small.index.map(str)
    l2c_small['id'] = file[:-9] + '_' + l2c_small.index.map(str)
    llc_small['id'] = file[:-9] + '_' + llc_small.index.map(str)
    l1d_accesses['id'] = file[:-9] + '_' + l1d_accesses.index.map(str)
    l2c_accesses['id'] = file[:-9] + '_' + l2c_accesses.index.map(str)
    llc_accesses['id'] = file[:-9] + '_' + llc_accesses.index.map(str)
    l1d_misses['id'] = file[:-9] + '_' + l1d_misses.index.map(str)
    l2c_misses['id'] = file[:-9] + '_' + l2c_misses.index.map(str)
    llc_misses['id'] = file[:-9] + '_' + llc_misses.index.map(str)
    
    l1d_full = pd.concat([l1d_full, l1d_small], ignore_index = True)
    l2c_full = pd.concat([l2c_full, l2c_small], ignore_index = True)
    llc_full = pd.concat([llc_full, llc_small], ignore_index = True)
    l1d_accesses_full = pd.concat([l1d_accesses_full, l1d_accesses], ignore_index = True)
    l2c_accesses_full = pd.concat([l2c_accesses_full, l2c_accesses], ignore_index = True)
    llc_accesses_full = pd.concat([llc_accesses_full, llc_accesses], ignore_index = True)
    l1d_misses_full = pd.concat([l1d_misses_full, l1d_misses], ignore_index = True)
    l2c_misses_full = pd.concat([l2c_misses_full, l2c_misses], ignore_index = True)
    llc_misses_full = pd.concat([llc_misses_full, llc_misses], ignore_index = True)
    # print(llc_misses_full)
 

|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

In [4]:
print(l2c_accesses_full)

                         id  1024  2048  4096  8192
0       400.perlbench-41B_0   994   994   994   994
1       400.perlbench-41B_1  1262  1262  1262  1262
2       400.perlbench-41B_2   761   761   761   761
3       400.perlbench-41B_3   464   464   464   464
4       400.perlbench-41B_4   831   831   831   831
...                     ...   ...   ...   ...   ...
141556     657.xz_s-56B_744   838   838   838   838
141557     657.xz_s-56B_745   836   836   836   836
141558     657.xz_s-56B_746   838   838   838   838
141559     657.xz_s-56B_747   836   836   836   836
141560     657.xz_s-56B_748   836   836   836   836

[141561 rows x 5 columns]


In [ ]:
print(l1d_accesses_full)
print(llc_full)

In [ ]:
cache_path = 'ship_cache'
l1d_full.to_csv(cache_path + '/l1d_miss_ratio.csv',index=False)
l2c_full.to_csv(cache_path + '/l2c_miss_ratio.csv',index=False)
llc_full.to_csv(cache_path + '/llc_miss_ratio.csv',index=False)
l1d_accesses_full.to_csv(cache_path + '/l1d_accesses.csv',index=False)
l2c_accesses_full.to_csv(cache_path + '/l2c_accesses.csv',index=False)
llc_accesses_full.to_csv(cache_path + '/llc_accesses.csv',index=False)
l1d_misses_full.to_csv(cache_path + '/l1d_misses.csv',index=False)
l2c_misses_full.to_csv(cache_path + '/l2c_misses.csv',index=False)
llc_misses_full.to_csv(cache_path + '/llc_misses.csv',index=False)

# Raw data -> writeback accesses, misses

In [ ]:
import json
import os
import csv
import pandas as pd
import numpy as np

In [ ]:
path = '../ce_raw_data/raw_data/'
id_ctr = 0
columns = ['id','32','64','128','256','512','1024','2048','4096','8192']
wb_accesses_full = pd.DataFrame()
wb_misses_full = pd.DataFrame()
for file in os.listdir(path):
    if file[-9:] != '_2048.out':
        continue
    wb_accesses = pd.DataFrame(0., index=np.arange(749), columns=columns)
    wb_misses = pd.DataFrame(0., index=np.arange(749), columns=columns)
    for c in columns[1:]:
        fname = file[:-9] + '_' + c + '.out'
        checker = False
        with open(path+fname) as f:
            lines = f.readlines()
            ctr = 0
            prev_misses_l1d = 0.
            prev_accesses_l1d = 0.
            prev_misses_wb = 0.
            prev_accesses_wb = 0.
            for line in lines:
                if line == "~~~~~~~~~~Reuse Distance Profiler output~~~~~~~~~~\n":
                    checker=True
                if line[:14] == "cpu0_L1D TOTAL":
                    ctr+=1
                if line[:13] == "LLC WRITEBACK":
                    if ctr == 1:
                        prev_misses_wb = int(line.split()[7])
                        prev_accesses_wb = int(line.split()[3])
                    if 751 > ctr and ctr > 1:
                        wb_misses[c][ctr-2] = int(line.split()[7]) - prev_misses_wb
                        prev_misses_wb = int(line.split()[7])
                        wb_accesses[c][ctr-2] = int(line.split()[3]) - prev_accesses_wb
                        prev_accesses_wb = int(line.split()[3])
            if not checker or ctr != 751:
                print("\n Error:", fname) # Check if a file didn't finish correctly
    print("|",end="")
    #print(l1d_small)
    wb_accesses['id'] = file[:-9] + '_' + wb_accesses.index.map(str)
    wb_misses['id'] = file[:-9] + '_' + wb_misses.index.map(str)
    
    #print(l1d_small)
    wb_accesses_full = pd.concat([wb_accesses_full, wb_accesses], ignore_index = True)
    wb_misses_full = pd.concat([wb_misses_full, wb_misses], ignore_index = True)
    print(wb_misses_full)

In [ ]:
wb_accesses_full.to_csv('caches/llc_writeback_accesses.csv',index=False)
wb_misses_full.to_csv('caches/llc_writeback_misses.csv',index=False)

In [5]:
import json
import os
import csv
import pandas as pd
import numpy as np

In [ ]:
path = '../ce_raw_data/raw_data/'
id_ctr = 0
columns = ['id','32','64','128','256','512','1024','2048','4096','8192']
wb_accesses_full = pd.DataFrame()
wb_misses_full = pd.DataFrame()
for file in os.listdir(path):
    if file[-9:] != '_2048.out':
        continue
    wb_accesses = pd.DataFrame(0., index=np.arange(749), columns=columns)
    wb_misses = pd.DataFrame(0., index=np.arange(749), columns=columns)
    for c in columns[1:]:
        fname = file[:-9] + '_' + c + '.out'
        checker = False
        with open(path+fname) as f:
            lines = f.readlines()
            ctr = 0
            prev_misses_l1d = 0.
            prev_accesses_l1d = 0.
            prev_misses_wb = 0.
            prev_accesses_wb = 0.
            for line in lines:
                if line == "~~~~~~~~~~Reuse Distance Profiler output~~~~~~~~~~\n":
                    checker=True
                if line[:14] == "cpu0_L1D TOTAL":
                    ctr+=1
                if line[:13] == "LLC WRITEBACK":
                    if ctr == 1:
                        prev_misses_wb = int(line.split()[7])
                        prev_accesses_wb = int(line.split()[3])
                    if 751 > ctr and ctr > 1:
                        wb_misses[c][ctr-2] = int(line.split()[7]) - prev_misses_wb
                        prev_misses_wb = int(line.split()[7])
                        wb_accesses[c][ctr-2] = int(line.split()[3]) - prev_accesses_wb
                        prev_accesses_wb = int(line.split()[3])
            if not checker or ctr != 751:
                print("\n Error:", fname) # Check if a file didn't finish correctly
    print("|",end="")
    #print(l1d_small)
    wb_accesses['id'] = file[:-9] + '_' + wb_accesses.index.map(str)
    wb_misses['id'] = file[:-9] + '_' + wb_misses.index.map(str)
    
    #print(l1d_small)
    wb_accesses_full = pd.concat([wb_accesses_full, wb_accesses], ignore_index = True)
    wb_misses_full = pd.concat([wb_misses_full, wb_misses], ignore_index = True)
    print(wb_misses_full)

# Raw data -> load + rfo accesses, misses

In [1]:
import json
import os
import csv
import pandas as pd
import numpy as np

In [10]:
path = '../ce_raw_data/l2c512/'
id_ctr = 0
#columns = ['id','1024','2048','4096','8192']
columns = ['id','2048','4096']
load_accesses_full = pd.DataFrame()
load_misses_full = pd.DataFrame()
rfo_accesses_full = pd.DataFrame()
rfo_misses_full = pd.DataFrame()
for file in os.listdir(path):
    if file[-9:] != '_2048.out':
        continue
    load_accesses = pd.DataFrame(0., index=np.arange(749), columns=columns)
    load_misses = pd.DataFrame(0., index=np.arange(749), columns=columns)
    rfo_accesses = pd.DataFrame(0., index=np.arange(749), columns=columns)
    rfo_misses = pd.DataFrame(0., index=np.arange(749), columns=columns)
    for c in columns[1:]:
        fname = file[:-9] + '_' + c + '.out'
        checker = False
        with open(path+fname) as f:
            lines = f.readlines()
            ctr = 0
            prev_misses_load = 0.
            prev_accesses_load = 0.
            prev_misses_rfo = 0.
            prev_accesses_rfo = 0.
            for line in lines:
                if line == "~~~~~~~~~~Reuse Distance Profiler output~~~~~~~~~~\n":
                    checker=True
                if line[:14] == "cpu0_L1D TOTAL":
                    ctr+=1
                if line[:8] == "LLC LOAD":
                    if ctr == 1:
                        prev_misses_load = int(line.split()[7])
                        prev_accesses_load = int(line.split()[3])
                    if 751 > ctr and ctr > 1:
                        load_misses[c][ctr-2] = int(line.split()[7]) - prev_misses_load
                        prev_misses_load = int(line.split()[7])
                        load_accesses[c][ctr-2] = int(line.split()[3]) - prev_accesses_load
                        prev_accesses_load = int(line.split()[3])
                if line[:7] == "LLC RFO":
                    if ctr == 1:
                        prev_misses_rfo = int(line.split()[7])
                        prev_accesses_rfo = int(line.split()[3])
                    if 751 > ctr and ctr > 1:
                        rfo_misses[c][ctr-2] = int(line.split()[7]) - prev_misses_rfo
                        prev_misses_rfo = int(line.split()[7])
                        rfo_accesses[c][ctr-2] = int(line.split()[3]) - prev_accesses_rfo
                        prev_accesses_rfo = int(line.split()[3])
            if not checker or ctr != 751:
                print("\n Error:", fname) # Check if a file didn't finish correctly
    print("|",end="")
    rfo_accesses['id'] = file[:-9] + '_' + rfo_accesses.index.map(str)
    rfo_misses['id'] = file[:-9] + '_' + rfo_misses.index.map(str)
    load_misses['id'] = file[:-9] + '_' + rfo_accesses.index.map(str)
    load_accesses['id'] = file[:-9] + '_' + rfo_misses.index.map(str)
    rfo_accesses_full = pd.concat([rfo_accesses_full, rfo_accesses], ignore_index = True)
    rfo_misses_full = pd.concat([rfo_misses_full, rfo_misses], ignore_index = True)
    load_accesses_full = pd.concat([load_accesses_full, load_accesses], ignore_index = True)
    load_misses_full = pd.concat([load_misses_full, load_misses], ignore_index = True)


|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

In [12]:
cache_path = 'l2c512'
rfo_accesses_full.to_csv(cache_path + '/llc_rfo_accesses.csv',index=False)
rfo_misses_full.to_csv(cache_path + '/llc_rfo_misses.csv',index=False)
load_accesses_full.to_csv(cache_path + '/llc_load_accesses.csv',index=False)
load_misses_full.to_csv(cache_path + '/llc_load_misses.csv',index=False)

In [11]:
print(load_accesses_full)

                         id   2048   4096
0       400.perlbench-41B_0  594.0  594.0
1       400.perlbench-41B_1  767.0  767.0
2       400.perlbench-41B_2  477.0  477.0
3       400.perlbench-41B_3  279.0  279.0
4       400.perlbench-41B_4  520.0  520.0
...                     ...    ...    ...
141556     657.xz_s-56B_744  838.0  838.0
141557     657.xz_s-56B_745  836.0  836.0
141558     657.xz_s-56B_746  838.0  838.0
141559     657.xz_s-56B_747  836.0  836.0
141560     657.xz_s-56B_748  836.0  836.0

[141561 rows x 3 columns]
